In [22]:
import csv
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [23]:
df = pd.read_csv("/home/drosophila-lab/Documents/Genomics Project/snp-data/SNP_CSV.csv")
print('Finished loading.')

df.head()

Finished loading.


,Chr,Pos,Sel,Pop,Evolving,Freq1,Freq2,Freq3,Freq4
0,X,221,1,CACO,False,0.852632,0.927083,0.901639,0.918129
1,X,221,2,CACO,False,0.853982,0.885106,0.883721,0.965714
2,X,221,3,CACO,False,0.975248,0.956098,0.903846,0.950495
3,X,221,4,CACO,False,0.814815,0.854962,0.941176,0.952586
4,X,221,5,CACO,False,0.873096,0.893939,0.870270,0.950893


In [24]:
df['Evolving'] = df['Evolving'].astype(int)

In [62]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
df_copy = df.copy() 

categorical_variables = df_copy[["Pos"]]

# Make LabelEncoder Object
label_encoder = LabelEncoder()

# Then fit and transform the whole categorical variable
label_encoded_features = label_encoder.fit_transform(categorical_variables)

# Once the features are label encoded, now we can perform OneHotEncoding
one_hot_encoder = OneHotEncoder()

# Then fit and transform the Label encoded features
one_hot_encoded = one_hot_encoder.fit_transform(label_encoded_features.reshape(1, -1))
#one_hot_encoded = one_hot_encoder.transform(one_hot_encoded1)

/home/drosophila-lab/Documents/Genomics Project/snp-data/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [63]:
#Create a Pandas DataFrame of the hot encoded column
one_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.categories_[0])
#concat with original data
df_copy = pd.concat([df_copy, one_df], axis=1).drop(['Pos'], axis=1)

In [25]:
print(df['Pop'].unique())

# one model is CACO, other is CAO, third is NACO, fourth is ANCO

['CACO' 'CAO' 'NACO' 'ANCO']


In [34]:
print(df['Sel'].unique())

[1 2 3 4 5]


In [38]:
df.groupby(['Pop','Sel']).size().reset_index().rename(columns={0:'count'})

,Pop,Sel,count
0,ANCO,1,1426593
1,ANCO,2,1426593
2,ANCO,3,1426593
3,ANCO,4,1426593
4,ANCO,5,1426593
5,CACO,1,1426593
6,CACO,2,1426593
7,CACO,3,1426593
8,CACO,4,1426593
9,CACO,5,1426593


In [64]:
features1 = df_copy[['Pos', 'Sel', 'Freq1', 'Freq2', 'Freq3', 'Freq4'] and df['Pop'] == 'CACO']
print(len(features1))

7132965


In [ ]:
features2 = df[['Pos', 'Sel', 'Freq1', 'Freq2', 'Freq3', 'Freq4'] and df['Pop'] == 'CAO']
print(len(features2))

7132965


In [ ]:
features3 = df[['Pos', 'Sel', 'Freq1', 'Freq2', 'Freq3', 'Freq4'] and df['Pop'] == 'ANCO']
print(len(features3))

7132965


In [ ]:
features4 = df[['Pos', 'Sel', 'Freq1', 'Freq2', 'Freq3', 'Freq4'] and df['Pop'] == 'NACO']
print(len(features4))

7132965


In [ ]:
featuresa = df[['Pos', 'Sel', 'Freq1', 'Freq2', 'Freq3', 'Freq4']]
print(len(featuresa))

28531860


In [ ]:
# Define features and label

features = features1
labels = df['Evolving']

# Drop NaNs or Infs
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features.dropna(inplace=True)
labels = labels.loc[features.index]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)
print('Finished splitting.')


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

: 

In [41]:
# Train model
print('Started training.')
clf = RandomForestClassifier(n_estimators=50, class_weight='balanced', random_state=42, verbose=1, n_jobs= 20)
clf.fit(X_train, y_train)
print('Finished training.')

Started training.


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.


KeyboardInterrupt: 

In [ ]:
# Evaluate
y_pred = clf.predict(X_test)
print(clf.feature_importances_)
print(classification_report(y_test, y_pred))

In [ ]:
import pickle

# save
with open('CACO_SNPs_model.pkl','wb') as f:
    pickle.dump(clf,f)


In [ ]:
# Load the model:
# with open('model.pkl', 'rb') as f:
#     clf2 = pickle.load(f)

# clf2.predict(X[0:1])